In [33]:
import pandas as pd
import numpy as np
import json
import calculations
import plotly.express as px
import plotly.subplots as sp
import plotly.graph_objects as go
import statsmodels.api as sm
import warnings
from scipy.stats import iqr, skew, kurtosis, pearsonr, ttest_ind

In [45]:
# Disable runtime warnings due to datasets containing NaNs after skew/kurtosis calcs
warnings.filterwarnings('ignore', category=RuntimeWarning)

# Process data

stimuli = []
outlier = []
scale_type = {
    "gesture_pitch_number": 100,
    "gesture_pitch_greyscale": 49,
    "gesture_roll_number": 100,
    "gesture_roll_greyscale": 49,
    "slider_number": 100,
    "slider_greyscale": 49,
}

outlier_cut_off = {
    "gesture_pitch_number": 50,
    "gesture_pitch_greyscale": 25,
    "gesture_roll_number": 50,
    "gesture_roll_greyscale": 25,
    "slider_number": 50,
    "slider_greyscale": 25,
}

truth_value_batches_100 = [
    [i for i in range(0, 10)],
    [i for i in range(10, 20)],
    [i for i in range(20, 30)],
    [i for i in range(30, 40)],
    [i for i in range(40, 50)],
    [i for i in range(50, 60)],
    [i for i in range(60, 70)],
    [i for i in range(70, 80)],
    [i for i in range(80, 90)],
    [i for i in range(90, 101)]
]

truth_value_batches_49 = [
    [i for i in range(0, 10)],
    [i for i in range(10, 20)],
    [i for i in range(20, 30)],
    [i for i in range(30, 40)],
    [i for i in range(40, 50)]
]

experiment_truth_type = {
    "gesture_pitch_number": {
        "truth": "pitch_truth",
        "truth_diff": "pitch_truth_diff",
        "label": "Pitch Truth",
        "input_type": "device"
    },
    "gesture_pitch_greyscale": {
        "truth": "pitch_truth",
        "truth_diff": "pitch_truth_diff",
        "label": "Pitch Truth",
        "input_type": "device"
    },
    "gesture_roll_number": {
        "truth": "roll_truth",
        "truth_diff": "roll_truth_diff",
        "label": "Roll Truth",
        "input_type": "device"
    },
    "gesture_roll_greyscale": {
        "truth": "roll_truth",
        "truth_diff": "roll_truth_diff",
        "label": "Roll Truth",
        "input_type": "device"
    },
    "slider_number": {
        "truth": "value",
        "truth_diff": "slider_diff",
        "label": "Slider Value",
        "input_type": "slider"
    },
    "slider_greyscale": {
        "truth": "value",
        "truth_diff": "slider_diff",
        "label": "Slider Value",
        "input_type": "slider"
    },
}

def ensure_not_outlier(value, experiment_type):
    cut_off = outlier_cut_off[experiment_type]
    return value < cut_off

with open('experiment_data/combined_participant_data.json', 'r') as file:
    data = json.load(file)

    if data:
        for participant in data:
            participant_id = participant.get("id")
            for experiment in participant["completedExperiments"]:
                experiment_type = experiment.get("experimentType")
                started_date = experiment.get("startedDate")
                ended_date = experiment.get("endedDate")
                for stimulus in experiment["successfulStimuli"]:
                    sensor_reading = stimulus.get("sensorReading", {})
                    quaternion = (
                        sensor_reading.get("w"),
                        sensor_reading.get("x"),
                        sensor_reading.get("y"),
                        sensor_reading.get("z")
                    )
                    # Remap quaternion -- the stored data from device app is in an incorrect order (actual values of w, x, y, z -> x, y, z, w)
                    x, y, z, w = quaternion
                    _, roll, pitch = calculations.quaternion_to_euler(x, y, z, w)
                    
                    sensor_reading["roll"] = roll
                    sensor_reading["pitch"] = pitch

                    pitch_scaled = calculations.scale_pitch(pitch, scale_type[experiment_type])
                    roll_scaled = calculations.scale_roll(roll, scale_type[experiment_type])
                    truth_value = stimulus.get("truth")
                    slider_value = stimulus.get("value")
                    
                    pitch_difference = abs(truth_value - pitch_scaled)
                    roll_difference = abs(truth_value - roll_scaled)

                    stimulus["participant_id"] = participant_id
                    stimulus["pitch"] = pitch
                    stimulus["pitch_truth"] = pitch_scaled
                    stimulus["pitch_truth_diff"] = pitch_difference
                
                    stimulus["roll"] = roll
                    stimulus["roll_truth"] = roll_scaled
                    stimulus["roll_truth_diff"] = roll_difference
                    
                    stimulus["slider_diff"] = abs(truth_value - slider_value)
                    
                    if truth_value == 0:
                        stimulus["pitch_truth_diff_percentage"] = (pitch_scaled / scale_type[experiment_type]) * 100
                        stimulus["roll_truth_diff_percentage"] = (roll_scaled / scale_type[experiment_type]) * 100
                    else:
                        stimulus["pitch_truth_diff_percentage"] = (pitch_difference / truth_value) * 100
                        stimulus["roll_truth_diff_percentage"] = (roll_difference / truth_value) * 100
                        
                    stimulus["experiment_type"] = experiment_type
                    stimulus["experiment_time_spent"] = ended_date - started_date
                    stimulus["calibration"] = sensor_reading.get("calibration_status")
                            
                    diff = stimulus[experiment_truth_type[experiment_type]["truth_diff"]]
                    if ensure_not_outlier(diff, experiment_type):
                        stimuli.append(stimulus)
                    else:
                        outlier.append(stimulus)

df = pd.DataFrame(data=stimuli)
o_df = pd.DataFrame(data=outlier)

In [46]:
display(df)
display(o_df)

,value,id,truth,inputType,sensorReading,participant_id,pitch,pitch_truth,pitch_truth_diff,roll,roll_truth,roll_truth_diff,slider_diff,pitch_truth_diff_percentage,roll_truth_diff_percentage,experiment_type,experiment_time_spent,calibration
0,50.782763,1,49,slider,"{'stability': 'In motion', 'activity': 'Still'...",1,-8.758405,10,39,-6.629249,7,42,1.782763,79.591837,85.714286,slider_number,183.397782,3
1,21.909768,2,32,slider,"{'y': 0.411926, 'activity_confidence': 92, 'x'...",1,-8.758405,10,22,-6.629249,7,25,10.090232,68.750000,78.125000,slider_number,183.397782,3
2,74.312516,3,76,slider,"{'y': 0.411926, 'timestamp': 1719216751, 'dura...",1,-8.758405,10,66,-6.629249,7,69,1.687484,86.842105,90.789474,slider_number,183.397782,3
3,1.492341,4,6,slider,"{'stability': 'In motion', 'z': 0.906189, 'cal...",1,-8.758405,10,4,-6.629249,7,1,4.507659,66.666667,16.666667,slider_number,183.397782,3
4,9.036517,5,26,slider,"{'y': 0.411926, 'timestamp': 1719216751, 'cali...",1,-8.758405,10,16,-6.629249,7,19,16.963483,61.538462,73.076923,slider_number,183.397782,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3378,0.000000,11,10,device,"{'duration': 190, 'y': 0.811279, 'activity': '...",33,41.192216,22,12,4.202357,2,8,10.000000,120.000000,80.000000,gesture_pitch_greyscale,170.829092,2
3379,0.000000,12,20,device,"{'calibration_status': 2, 'activity_confidence...",33,4.643358,3,17,-2.629373,1,19,20.000000,85.000000,95.000000,gesture_pitch_greyscale,170.829092,2
3380,0.000000,13,27,device,"{'y': 0.820068, 'stability': 'Stable', 'activi...",33,40.459716,22,5,4.699984,3,24,27.000000,18.518519,88.888889,gesture_pitch_greyscale,170.829092,2
3381,0.000000,14,7,device,"{'activity_confidence': 97, 'activity': 'Still...",33,41.920006,23,16,3.326768,2,5,7.000000,228.571429,71.428571,gesture_pitch_greyscale,170.829092,2


,inputType,truth,sensorReading,value,id,participant_id,pitch,pitch_truth,pitch_truth_diff,roll,roll_truth,roll_truth_diff,slider_diff,pitch_truth_diff_percentage,roll_truth_diff_percentage,experiment_type,experiment_time_spent,calibration
0,device,49,"{'z': 0.907104, 'activity': 'Still', 'activity...",0.0,2,1,32.076862,17,32,-5.646269,3,46,49.0,65.306122,93.877551,gesture_pitch_greyscale,145.304421,3
1,device,28,"{'timestamp': 1719217020, 'calibration_status'...",0.0,7,1,2.818003,2,26,-12.016960,7,21,28.0,92.857143,75.000000,gesture_pitch_greyscale,145.304421,3
2,device,44,"{'x': -0.206421, 'activity_confidence': 82, 'w...",0.0,5,1,-14.976900,8,36,-26.702153,15,29,44.0,81.818182,65.909091,gesture_roll_greyscale,181.775959,2
3,device,4,"{'activity_confidence': 93, 'activity': 'Still...",0.0,12,1,140.093331,49,45,-89.010138,48,44,4.0,1125.000000,1100.000000,gesture_roll_greyscale,181.775959,2
4,device,33,"{'activity': 'Still', 'y': 0.888, 'z': 0.42010...",0.0,13,1,-17.034583,9,24,-13.425065,7,26,33.0,72.727273,78.787879,gesture_roll_greyscale,181.775959,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
572,device,33,"{'calibration_status': 2, 'activity_confidence...",0.0,15,33,4.668507,3,30,-1.623507,1,32,33.0,90.909091,96.969697,gesture_pitch_greyscale,170.829092,2
573,device,0,"{'stability': 'Stable', 'z': 0.442078, 'w': 0....",0.0,16,33,49.661682,27,27,3.824130,2,2,0.0,55.102041,4.081633,gesture_pitch_greyscale,170.829092,2
574,device,47,"{'y': 0.879944, 'stability': 'Stable', 'x': 0....",0.0,17,33,2.677436,1,46,-1.854517,1,46,47.0,97.872340,97.872340,gesture_pitch_greyscale,170.829092,2
575,device,3,"{'duration': 210, 'activity_confidence': 97, '...",0.0,18,33,76.534100,42,39,4.726522,3,0,3.0,1300.000000,0.000000,gesture_pitch_greyscale,170.829092,2


In [36]:
calibration_threshold = 2
g_pitch_greyscale_df = df[(df["experiment_type"] == "gesture_pitch_greyscale") & (df["calibration"] >= calibration_threshold)]
g_pitch_number_df = df[(df["experiment_type"] == "gesture_pitch_number") & (df["calibration"] >= calibration_threshold)]
g_roll_greyscale_df = df[(df["experiment_type"] == "gesture_roll_greyscale") & (df["calibration"] >= calibration_threshold)]
g_roll_number_df = df[(df["experiment_type"] == "gesture_roll_number") & (df["calibration"] >= calibration_threshold)]
g_slider_greyscale_df = df[(df["experiment_type"] == "slider_greyscale")]
g_slider_number_df = df[(df["experiment_type"] == "slider_number")]

fig = sp.make_subplots(rows=6, cols=2, subplot_titles=(
    'Pitch - Greyscale (Scaled 0-49)',
    'Pitch Error - Greyscale (Scaled 0-49)',
    'Pitch - Number (Scaled 0-100)',
    'Pitch Error - Number (Scaled 0-100)',
    'Roll - Greyscale (Scaled 0-49)',
    'Roll Error - Greyscale (Scaled 0-49)',
    'Roll - Number (Scaled 0-100)',
    'Roll Error - Number (Scaled 0-100)',
    'Slider - Greyscale (Scaled 0-49)',
    'Slider Error - Greyscale (Scaled 0-49)',
    'Slider - Number (Scaled 0-100)',
    'Slider Error - Number (Scaled 0-100)'
))

fig.add_trace(px.scatter(g_pitch_greyscale_df, x="truth", y="pitch_truth", color_discrete_sequence=['blue']).data[0], row=1, col=1)
fig.add_trace(px.scatter(g_pitch_greyscale_df, x="truth", y="pitch_truth", error_y="pitch_truth_diff", color_discrete_sequence=['blue']).data[0], row=1, col=2)
fig.add_trace(px.scatter(g_pitch_number_df, x="truth", y="pitch_truth", color_discrete_sequence=['green']).data[0], row=2, col=1)
fig.add_trace(px.scatter(g_pitch_number_df, x="truth", y="pitch_truth", error_y="pitch_truth_diff", color_discrete_sequence=['green']).data[0], row=2, col=2)
fig.add_trace(px.scatter(g_roll_greyscale_df, x="truth", y="roll_truth", color_discrete_sequence=['red']).data[0], row=3, col=1)
fig.add_trace(px.scatter(g_roll_greyscale_df, x="truth", y="roll_truth", error_y="roll_truth_diff", color_discrete_sequence=['red']).data[0], row=3, col=2)
fig.add_trace(px.scatter(g_roll_number_df, x="truth", y="roll_truth", color_discrete_sequence=['purple']).data[0], row=4, col=1)
fig.add_trace(px.scatter(g_roll_number_df, x="truth", y="roll_truth", error_y="roll_truth_diff", color_discrete_sequence=['purple']).data[0], row=4, col=2)
fig.add_trace(px.scatter(g_slider_greyscale_df, x="truth", y="value", color_discrete_sequence=['magenta']).data[0], row=5, col=1)
fig.add_trace(px.scatter(g_slider_greyscale_df, x="truth", y="value", error_y="slider_diff", color_discrete_sequence=['magenta']).data[0], row=5, col=2)
fig.add_trace(px.scatter(g_slider_number_df, x="truth", y="value", color_discrete_sequence=['purple']).data[0], row=6, col=1)
fig.add_trace(px.scatter(g_slider_number_df, x="truth", y="value", error_y="slider_diff", color_discrete_sequence=['purple']).data[0], row=6, col=2)

fig.update_layout(height=2000, width=1600, title_text="Sensor Readings Scatter Plots")
fig.update_xaxes(title_text="Truth Values")
fig.update_yaxes(title_text="Sensor Readings")

fig['layout']['yaxis1'].update(title='Pitch Sensor Readings')
fig['layout']['yaxis2'].update(title='Pitch Sensor Readings')
fig['layout']['yaxis3'].update(title='Pitch Sensor Readings')
fig['layout']['yaxis4'].update(title='Pitch Sensor Readings')
fig['layout']['yaxis5'].update(title='Roll Sensor Readings')
fig['layout']['yaxis6'].update(title='Roll Sensor Readings')
fig['layout']['yaxis7'].update(title='Roll Sensor Readings')
fig['layout']['yaxis8'].update(title='Roll Sensor Readings')
fig['layout']['yaxis9'].update(title='Slider Readings')
fig['layout']['yaxis10'].update(title='Slider Readings')
fig['layout']['yaxis11'].update(title='Slider Readings')
fig['layout']['yaxis12'].update(title='Slider Readings')

fig.show()

In [37]:
def create_deviation_plots(truth_values, experiment_type, truth_type):
    filtered_df = df[(df['experiment_type'] == experiment_type) & 
                     (df['truth'].isin(truth_values)) & 
                     (df['inputType'] == truth_type["input_type"])]

    mean_truth = filtered_df.groupby("truth")[truth_type["truth"]].mean().reset_index()
    std_truth = filtered_df.groupby("truth")[truth_type["truth"]].std().reset_index()

    mean_std_df = pd.merge(mean_truth, std_truth, on="truth", suffixes=('_mean', '_std'))
    
    # Have to fill NaNs with zeroes, needed if one or more truths only have one reading
    mean_std_df[f'{truth_type["truth"]}_std'] = mean_std_df[f'{truth_type["truth"]}_std'].fillna(0)
    mean_std_df[f'{truth_type["truth"]}_std'] = mean_std_df[f'{truth_type["truth"]}_std'].abs()
    
    fig = go.Figure()
    fig.add_trace(go.Scatter(
        x=mean_std_df["truth"], y=mean_std_df[f'{truth_type["truth"]}_mean'],
        mode='lines', line=dict(color='blue'), name=f'Mean {truth_type["label"]}'
    ))

    upper_bound = mean_std_df[f'{truth_type["truth"]}_mean'] + mean_std_df[f'{truth_type["truth"]}_std']
    lower_bound = mean_std_df[f'{truth_type["truth"]}_mean'] - mean_std_df[f'{truth_type["truth"]}_std']
    
    fig.add_trace(go.Scatter(
        x=mean_std_df["truth"], y=upper_bound,
        fill=None, mode='lines', line=dict(color='lightblue'), name='Upper Deviation Range', showlegend=False
    ))
    fig.add_trace(go.Scatter(
        x=mean_std_df["truth"], y=lower_bound,
        fill='tonexty', mode='lines', line=dict(color='lightblue'), name='Deviation Range'
    ))

    fig.update_layout(
        title=f"Deviation Plot for {truth_type['label']} - {experiment_type}",
        xaxis_title="Truth Values",
        yaxis_title=f"{truth_type['label']} Values",
        hovermode="x"
    )

    fig.show()
    
create_deviation_plots([i for i in range(0, 101)], "gesture_pitch_number", experiment_truth_type["gesture_pitch_number"])
    
for batch in truth_value_batches_100:
    create_deviation_plots(batch, "gesture_pitch_number", experiment_truth_type["gesture_pitch_number"])
    
create_deviation_plots([i for i in range(0, 50)], "gesture_pitch_greyscale", experiment_truth_type["gesture_pitch_greyscale"])

for batch in truth_value_batches_49:
    create_deviation_plots(batch, "gesture_pitch_greyscale", experiment_truth_type["gesture_pitch_greyscale"])
    
create_deviation_plots([i for i in range(0, 101)], "gesture_roll_number", experiment_truth_type["gesture_roll_number"])

for batch in truth_value_batches_100:
    create_deviation_plots(batch, "gesture_roll_number", experiment_truth_type["gesture_roll_number"])

create_deviation_plots([i for i in range(0, 50)], "gesture_roll_greyscale", experiment_truth_type["gesture_roll_greyscale"])

for batch in truth_value_batches_49:
    create_deviation_plots(batch, "gesture_roll_greyscale", experiment_truth_type["gesture_roll_greyscale"])
 
create_deviation_plots([i for i in range(0, 101)], "slider_number", experiment_truth_type["slider_number"])
    
for batch in truth_value_batches_100:
    create_deviation_plots(batch, "slider_number", experiment_truth_type["slider_number"])

create_deviation_plots([i for i in range(0, 50)], "slider_greyscale", experiment_truth_type["slider_greyscale"])

for batch in truth_value_batches_49:
    create_deviation_plots(batch, "slider_greyscale", experiment_truth_type["slider_greyscale"])

In [38]:
def create_histogram_truth_plots(experiment_type, truth_type):
    filtered_df = df[(df['experiment_type'] == experiment_type) & 
                     (df['inputType'] == truth_type["input_type"])]

    fig = px.histogram(filtered_df, x=truth_type["truth_diff"],
                    title=f'{truth_type['label']} Differences - {experiment_type}',
                    labels={f'{truth_type["truth_diff"]}': f'{truth_type["label"]} Difference'})
    
    fig.show()
    
for experiment_type, truth_type in experiment_truth_type.items():
    create_histogram_truth_plots(experiment_type, truth_type)

In [39]:
def create_histogram_plots(truth_values, experiment_type, truth_type):
    filtered_df = df[(df['experiment_type'] == experiment_type) & 
                     (df['truth'].isin(truth_values)) & 
                     (df['inputType'] == truth_type["input_type"])]

    filtered_df = filtered_df.sort_values(by='truth')

    fig = px.histogram(filtered_df, x=truth_type["truth"], color="truth", 
                       facet_col="truth", marginal="rug", 
                       labels={f'{truth_type["truth"]}': f'{truth_type["label"]}', 'truth': 'Truth'},
                       title=f'Distribution of {truth_type["label"]} Values for Truth Values {truth_values[0]} to {truth_values[-1]} - {experiment_type}',
                       nbins=int(filtered_df[truth_type["truth"]].max() - filtered_df[truth_type["truth"]].min() + 1))

    fig.show()
    
for batch in truth_value_batches_100:
    create_histogram_plots(batch, "gesture_pitch_number", experiment_truth_type["gesture_pitch_number"])
    
for batch in truth_value_batches_49:
    create_histogram_plots(batch, "gesture_pitch_greyscale", experiment_truth_type["gesture_pitch_greyscale"])
    
for batch in truth_value_batches_100:
    create_histogram_plots(batch, "gesture_roll_number", experiment_truth_type["gesture_roll_number"])
    
for batch in truth_value_batches_49:
    create_histogram_plots(batch, "gesture_roll_greyscale", experiment_truth_type["gesture_roll_greyscale"])

In [40]:
def calculate_summary_statistics(experiment_type, truth_type):
    filtered_df = df[df['experiment_type'] == experiment_type]
    grouped = filtered_df.groupby('truth')[truth_type].agg(['mean', 'std', 'min', 'max', 'count', 'median'])
    grouped['iqr'] = filtered_df.groupby('truth')[truth_type].apply(iqr)
    grouped['skew'] = filtered_df.groupby('truth')[truth_type].apply(skew)
    grouped['kurtosis'] = filtered_df.groupby('truth')[truth_type].apply(kurtosis)
    
    # Kurtosis and skew calculate NaNs on certain truths (0 and 100), but it's not an issue. Filling NaNs with zeroes
    grouped = grouped.fillna(0)
    return grouped.reset_index()

def generate_summary_tables():
    summary_tables = {}
    for experiment_type, truth_info in experiment_truth_type.items():
        summary_table = calculate_summary_statistics(experiment_type, truth_info['truth'])
        summary_table['experiment_type'] = experiment_type
        summary_tables[experiment_type] = summary_table
    return summary_tables

def calculate_truth_diff_percentages(experiment_type, truth_diff_column):
    filtered_df = df[df['experiment_type'] == experiment_type]
    count = filtered_df.shape[0]
    within_5 = (filtered_df[truth_diff_column].abs() <= 5).sum() / count * 100
    within_10 = (filtered_df[truth_diff_column].abs() <= 10).sum() / count * 100
    within_15 = (filtered_df[truth_diff_column].abs() <= 15).sum() / count * 100
    within_20 = (filtered_df[truth_diff_column].abs() <= 20).sum() / count * 100
    within_25 = (filtered_df[truth_diff_column].abs() <= 25).sum() / count * 100
    return within_5, within_10, within_15, within_20, within_25

def calculate_averages_per_experiment_type(summary_tables):
    averages = []
    for experiment_type, summary_table in summary_tables.items():
        within_5, within_10, within_15, within_20, within_25 = calculate_truth_diff_percentages(experiment_type, experiment_truth_type[experiment_type]['truth_diff'])
        
        time_spent_df = df[df['experiment_type'] == experiment_type].drop_duplicates(subset=['participant_id', 'experiment_type'])
        time_spent_avg = time_spent_df['experiment_time_spent'].abs().mean()
        time_spent_min = time_spent_df['experiment_time_spent'].abs().min()
        time_spent_max = time_spent_df['experiment_time_spent'].abs().max()
        
        averages.append({
            'experiment_type': experiment_type,
            'time_spent_avg': time_spent_avg,
            'time_spent_min': time_spent_min,
            'time_spent_max': time_spent_max,
            'mean': summary_table['mean'].mean(),
            'std': summary_table['std'].mean(),
            'min': summary_table['min'].min(),
            'max': summary_table['max'].max(),
            'count': summary_table['count'].count(),
            'median': summary_table['median'].median(),
            'iqr': summary_table['iqr'].mean(),
            'skew': summary_table['skew'].mean(),
            'kurtosis': summary_table['kurtosis'].mean(),
            'within_5_percent': within_5,
            'within_10_percent': within_10,
            'within_15_percent': within_15,
            'within_20_percent': within_20,
            'within_25_percent': within_25
        })
    return pd.DataFrame(averages)

summary_tables = generate_summary_tables()
averages_df = calculate_averages_per_experiment_type(summary_tables)

display(averages_df)

for _, summary_table in summary_tables.items():
    display(summary_table)

,experiment_type,time_spent_avg,time_spent_min,time_spent_max,mean,std,min,max,count,median,iqr,skew,kurtosis,within_5_percent,within_10_percent,within_15_percent,within_20_percent,within_25_percent
0,gesture_pitch_number,143.881667,73.162797,399.222038,44.554248,10.109810,0.0,100.0,101,46.500000,10.252475,0.196277,-0.774312,38.235294,63.467492,78.173375,88.390093,93.808050
1,gesture_pitch_greyscale,152.724591,100.113338,303.560128,22.312100,11.123101,0.0,49.0,50,21.500000,16.435000,0.178134,-1.042151,27.429806,46.436285,68.466523,86.177106,100.000000
2,gesture_roll_number,152.073224,80.542917,437.243360,51.989603,11.397014,0.0,98.0,101,54.000000,11.834158,-0.180755,-0.778534,27.480916,47.786260,67.328244,80.305344,88.396947
3,gesture_roll_greyscale,181.509191,103.378948,501.023651,24.907270,9.736131,0.0,48.0,50,25.000000,13.850000,-0.035477,-0.885375,25.502008,49.799197,71.084337,86.546185,100.000000
4,slider_number,160.304255,65.096681,313.937267,48.921237,3.509170,0.0,100.0,101,50.108070,3.813159,-0.086402,-0.695252,74.316109,93.465046,98.176292,99.240122,99.696049
5,slider_greyscale,158.260002,93.029034,246.268256,23.951604,11.668611,0.0,49.0,50,25.605878,17.587190,0.016843,-1.048047,27.429806,50.323974,69.978402,84.233261,100.000000


,truth,mean,std,min,max,count,median,iqr,skew,kurtosis,experiment_type
0,0,4.000000,4.444097,0,15,9,3.0,1.00,1.812662,2.464749,gesture_pitch_number
1,1,3.750000,4.349329,0,10,4,2.5,3.25,0.868396,-0.847173,gesture_pitch_number
2,2,5.000000,2.160247,2,7,4,5.5,2.00,-0.687243,-1.000000,gesture_pitch_number
3,3,3.000000,2.449490,1,6,4,2.5,3.50,0.314270,-1.592593,gesture_pitch_number
4,4,7.250000,4.494947,2,14,12,5.5,7.50,0.418334,-1.413246,gesture_pitch_number
...,...,...,...,...,...,...,...,...,...,...,...
96,96,91.000000,7.456541,83,100,6,89.0,12.25,0.351951,-1.542777,gesture_pitch_number
97,97,88.250000,11.324752,73,100,4,90.0,9.75,-0.495999,-1.018157,gesture_pitch_number
98,98,86.888889,9.752493,72,100,9,87.0,14.00,-0.290639,-1.150153,gesture_pitch_number
99,99,84.000000,9.230385,74,96,6,80.5,13.00,0.488085,-1.445745,gesture_pitch_number


,truth,mean,std,min,max,count,median,iqr,skew,kurtosis,experiment_type
0,0,11.444444,9.369677,0,24,18,12.5,17.25,0.056255,-1.670679,gesture_pitch_greyscale
1,1,11.125000,5.409978,2,18,8,12.5,7.50,-0.443529,-1.004141,gesture_pitch_greyscale
2,2,9.125000,9.478359,0,23,8,6.5,14.25,0.399079,-1.451608,gesture_pitch_greyscale
3,3,9.500000,9.648834,1,26,6,7.0,11.00,0.798988,-0.654596,gesture_pitch_greyscale
4,4,13.166667,10.553041,0,27,6,17.0,12.00,-0.236274,-1.273260,gesture_pitch_greyscale
5,5,12.888889,8.950481,0,29,9,12.0,8.00,0.339973,-0.547140,gesture_pitch_greyscale
6,6,11.500000,3.872983,6,15,4,12.5,3.00,-0.795046,-0.885926,gesture_pitch_greyscale
7,7,17.714286,10.419761,0,28,7,23.0,9.50,-0.891037,-0.839928,gesture_pitch_greyscale
8,8,11.000000,10.449880,1,29,11,6.0,16.00,0.645208,-1.126706,gesture_pitch_greyscale
9,9,14.000000,9.233093,6,33,9,11.0,4.00,1.254203,0.074703,gesture_pitch_greyscale


,truth,mean,std,min,max,count,median,iqr,skew,kurtosis,experiment_type
0,0,14.333333,18.929694,1,36,3,6.0,17.50,0.652012,-1.500000,gesture_roll_number
1,1,14.800000,9.391486,1,25,5,13.0,9.00,-0.406706,-0.999479,gesture_roll_number
2,2,19.000000,7.905694,4,28,9,21.0,10.00,-0.766345,-0.561072,gesture_roll_number
3,3,20.142857,10.007140,6,38,7,20.0,8.50,0.482046,-0.208409,gesture_roll_number
4,4,25.750000,5.560276,22,34,4,23.5,3.75,1.080610,-0.723084,gesture_roll_number
...,...,...,...,...,...,...,...,...,...,...,...
96,96,84.000000,14.899664,47,98,14,90.0,15.00,-1.350412,0.773910,gesture_roll_number
97,97,80.333333,17.048949,54,94,6,88.5,23.00,-0.735559,-1.219921,gesture_roll_number
98,98,78.200000,13.608821,60,94,5,76.0,17.00,-0.119054,-1.294905,gesture_roll_number
99,99,86.000000,13.928388,62,98,5,90.0,4.00,-1.221431,-0.059996,gesture_roll_number


,truth,mean,std,min,max,count,median,iqr,skew,kurtosis,experiment_type
0,0,11.000000,6.937218,1,22,17,12.0,13.00,0.011578,-1.422031,gesture_roll_greyscale
1,1,17.222222,7.870479,6,25,9,19.0,13.00,-0.508667,-1.387468,gesture_roll_greyscale
2,2,14.142857,7.425824,4,25,7,12.0,10.00,0.122738,-1.225167,gesture_roll_greyscale
3,3,18.142857,5.956189,9,24,7,19.0,9.50,-0.352918,-1.347549,gesture_roll_greyscale
4,4,19.250000,12.284814,1,27,4,24.5,8.75,-1.090483,-0.720551,gesture_roll_greyscale
5,5,16.400000,7.662318,2,27,10,16.0,8.75,-0.184514,-0.471391,gesture_roll_greyscale
6,6,17.000000,8.996296,4,30,16,13.5,14.25,0.180998,-1.366580,gesture_roll_greyscale
7,7,19.777778,5.607535,10,29,9,19.0,7.00,-0.010675,-0.481370,gesture_roll_greyscale
8,8,14.400000,8.384510,4,25,5,17.0,10.00,-0.062481,-1.342380,gesture_roll_greyscale
9,9,21.666667,8.194603,4,33,12,23.0,6.50,-0.833921,0.070827,gesture_roll_greyscale


,truth,mean,std,min,max,count,median,iqr,skew,kurtosis,experiment_type
0,0,0.000000,0.000000,0.000000,0.000000,7,0.000000,0.000000,0.000000,0.000000,slider_number
1,1,0.274649,0.329813,0.000000,0.952432,8,0.172917,0.326624,1.170934,0.187467,slider_number
2,2,0.504782,0.547771,0.062684,1.238188,4,0.359127,0.667964,0.579703,-1.262379,slider_number
3,3,1.543033,1.263062,0.121591,3.960529,7,1.289665,1.212593,0.932448,-0.010380,slider_number
4,4,1.438317,0.383883,0.939227,2.096687,7,1.296690,0.343714,0.598814,-0.613947,slider_number
...,...,...,...,...,...,...,...,...,...,...,...
96,96,97.407413,2.354941,94.162639,99.486481,4,97.990266,2.446783,-0.673280,-1.083310,slider_number
97,97,98.535673,0.705188,97.511674,99.248752,8,98.877689,1.096923,-0.537688,-1.415678,slider_number
98,98,99.307784,0.324006,99.078677,99.536891,2,99.307784,0.229107,0.000000,-2.000000,slider_number
99,99,99.702126,0.286189,99.313720,99.953773,4,99.770505,0.317959,-0.615287,-1.166082,slider_number


,truth,mean,std,min,max,count,median,iqr,skew,kurtosis,experiment_type
0,0,7.334519,9.717180,0.000000,23.779242,7,0.953980,13.304206,0.752715,-1.012503,slider_greyscale
1,1,3.669683,3.371034,0.000000,7.371508,5,4.133313,5.991251,-0.087321,-1.700845,slider_greyscale
2,2,11.477885,9.048226,0.000000,24.858178,10,12.836837,14.646608,-0.064499,-1.361582,slider_greyscale
3,3,5.686550,4.357942,1.275259,13.260913,6,5.882650,3.694565,0.728339,-0.351563,slider_greyscale
4,4,6.297403,8.924821,0.000000,27.240539,15,1.390460,9.617707,1.287846,0.283747,slider_greyscale
5,5,7.909330,9.532654,0.000000,28.889381,11,2.956681,11.559640,1.057357,0.038796,slider_greyscale
6,6,21.996382,9.331482,5.369027,30.598727,6,23.549942,9.145151,-0.924425,-0.260426,slider_greyscale
7,7,9.595542,9.026056,0.000000,22.779536,11,7.218135,16.859639,0.358054,-1.547832,slider_greyscale
8,8,4.885801,8.731085,0.000000,25.443989,9,0.000000,7.119905,1.647127,1.443800,slider_greyscale
9,9,11.196442,11.647316,0.000000,30.606294,11,4.436090,18.416245,0.621268,-1.241230,slider_greyscale


In [41]:
def create_and_analyze_plots(experiment_type, truth_info):
    filtered_df = df[(df['experiment_type'] == experiment_type) & 
                     (df['inputType'] == truth_info['input_type'])]

    if filtered_df.empty:
        return

    filtered_df = filtered_df.sort_values(by=['participant_id', 'id'])

    filtered_df['previous_truth'] = filtered_df.groupby('participant_id')[truth_info['truth']].shift(1)
    filtered_df['truth_prev_diff'] = abs(filtered_df[truth_info['truth']] - filtered_df['previous_truth'])

    filtered_df['deviation'] = abs(filtered_df[truth_info['truth']] - filtered_df['truth'])
    filtered_df['next_deviation'] = filtered_df.groupby('participant_id')['deviation'].shift(-1)

    fig1 = px.scatter(filtered_df, x='previous_truth', y=truth_info['truth'], trendline='lowess',
                      title=f'Learning Bias in {truth_info["label"]} Responses - {experiment_type}', trendline_color_override='black')
    fig1.show()

    fig2 = px.scatter(filtered_df, x='deviation', y='next_deviation', trendline='lowess',
                      title=f'Impact of {truth_info["label"]} Deviation on Next Response - {experiment_type}', trendline_color_override='black')
    fig2.show()

    valid_pairs_prev = filtered_df[['previous_truth', truth_info['truth']]].dropna()
    if not valid_pairs_prev.empty:
        prev_corr, prev_pval = pearsonr(valid_pairs_prev['previous_truth'], valid_pairs_prev[truth_info['truth']])
        prev_tstat, prev_tpval = ttest_ind(valid_pairs_prev['previous_truth'], valid_pairs_prev[truth_info['truth']])
        print(f"Correlation between previous and current {truth_info['label']} for {experiment_type}: {prev_corr}, p-value: {prev_pval}")
        print(f"t-statistic between previous and current {truth_info['label']} for {experiment_type}: {prev_tstat}, p-value: {prev_tpval}")

    valid_pairs_dev = filtered_df[['deviation', 'next_deviation']].dropna()
    if not valid_pairs_dev.empty:
        dev_corr, dev_pval = pearsonr(valid_pairs_dev['deviation'], valid_pairs_dev['next_deviation'])
        dev_tstat, dev_tpval = ttest_ind(valid_pairs_dev['deviation'], valid_pairs_dev['next_deviation'])
        print(f"Correlation between {truth_info['label']} deviation and next response for {experiment_type}: {dev_corr}, p-value: {dev_pval}")
        print(f"t-statistic between {truth_info['label']} deviation and next response for {experiment_type}: {dev_tstat}, p-value: {dev_tpval}")

create_and_analyze_plots("gesture_pitch_number", experiment_truth_type["gesture_pitch_number"])
create_and_analyze_plots("gesture_pitch_greyscale", experiment_truth_type["gesture_pitch_greyscale"])
create_and_analyze_plots("gesture_roll_number", experiment_truth_type["gesture_roll_number"])
create_and_analyze_plots("gesture_roll_greyscale", experiment_truth_type["gesture_roll_greyscale"])
create_and_analyze_plots("slider_number", experiment_truth_type["slider_number"])
create_and_analyze_plots("slider_greyscale", experiment_truth_type["slider_greyscale"])

Correlation between previous and current Pitch Truth for gesture_pitch_number: 0.15166804802542483, p-value: 0.00016371479774730945
t-statistic between previous and current Pitch Truth for gesture_pitch_number: -0.36284529820019323, p-value: 0.7167830979373275
Correlation between Pitch Truth deviation and next response for gesture_pitch_number: 0.4130742468378591, p-value: 1.166526324635214e-26
t-statistic between Pitch Truth deviation and next response for gesture_pitch_number: -0.15543791521604247, p-value: 0.8765017244023069


Correlation between previous and current Pitch Truth for gesture_pitch_greyscale: 0.30182749878434506, p-value: 1.656212398850351e-10
t-statistic between previous and current Pitch Truth for gesture_pitch_greyscale: -0.106343663434494, p-value: 0.9153345477114644
Correlation between Pitch Truth deviation and next response for gesture_pitch_greyscale: 0.00954116348816796, p-value: 0.8436097188792531
t-statistic between Pitch Truth deviation and next response for gesture_pitch_greyscale: -0.17125705537333413, p-value: 0.8640620259876565


Correlation between previous and current Roll Truth for gesture_roll_number: 0.14775297268024257, p-value: 0.00021743259857504257
t-statistic between previous and current Roll Truth for gesture_roll_number: 0.1461559653000292, p-value: 0.8838219844302593
Correlation between Roll Truth deviation and next response for gesture_roll_number: 0.2519254064509786, p-value: 1.8483440476393718e-10
t-statistic between Roll Truth deviation and next response for gesture_roll_number: 0.2035871190933015, p-value: 0.8387095101982607


Correlation between previous and current Roll Truth for gesture_roll_greyscale: 0.28128437987719823, p-value: 6.643707570017228e-10
t-statistic between previous and current Roll Truth for gesture_roll_greyscale: -0.08111777998806655, p-value: 0.9353657766630998
Correlation between Roll Truth deviation and next response for gesture_roll_greyscale: -0.043982721726861254, p-value: 0.3439739312916372
t-statistic between Roll Truth deviation and next response for gesture_roll_greyscale: -0.3970325063472747, p-value: 0.691434781978794


Correlation between previous and current Slider Value for slider_number: -0.05438363429462967, p-value: 0.174503612542553
t-statistic between previous and current Slider Value for slider_number: -0.17120051214087106, p-value: 0.8640938576620844
Correlation between Slider Value deviation and next response for slider_number: 0.19983494806896301, p-value: 4.734713345782376e-07
t-statistic between Slider Value deviation and next response for slider_number: 0.003222383763810781, p-value: 0.9974294291886159


Correlation between previous and current Slider Value for slider_greyscale: 0.139320335334408, p-value: 0.0037948681742623618
t-statistic between previous and current Slider Value for slider_greyscale: 0.287507900889586, p-value: 0.7737928784951394
Correlation between Slider Value deviation and next response for slider_greyscale: -0.015767817196796402, p-value: 0.7443964192217156
t-statistic between Slider Value deviation and next response for slider_greyscale: 0.304195089955881, p-value: 0.7610530249478041
